In [ ]:
%load_ext autoreload
%autoreload 2

import os
if 'notebooks' not in os.listdir():
    # want to go up one level
    os.chdir('..')

from collections import Counter

import numpy as np
import pandas as pd
from sklearn.metrics import f1_score
    
import util
from evaluation.downstream_task_evaluation import TASK_QUESTION_MAP
from evaluation.surface_level_probing import *
from evaluation.surface_level_probing import _build_functionality_whitelist, _build_location_whitelist
from features.map_response_to_bucket import map_to_bucket, get_bucket_mapping_fn
from util.prep_location_embeddings import get_location_embeddings

In [ ]:
def _filter_and_balance_dataset(student_data, task):
    reduction_function = None
    inclusion_function = None
    if task == 'depression' or task == 'sleep' or task == 'gpa' or 'depression' in task:
        reduction_function = get_bucket_mapping_fn(task)
    if task == 'class_year':
        inclusion_function = (lambda x: util.num_text_map[TASK_QUESTION_MAP[task]].get(str(x), None) in
                                        ['Freshman', 'Sophomore', 'Junior', 'Senior'])
    if task == 'gender':
        # unfortunately there is not enough data to perform classification outside of female/male
        inclusion_function = (
            lambda x: util.num_text_map[TASK_QUESTION_MAP[task]].get(str(x), None) in ['Female', 'Male'])
    if task == 'school':
        # a little bit unclear which schools should be including - this list has all with > 10 datapoints
        inclusion_function = (lambda x: x in ['Undergraduate L S & A', 'Undergraduate Engineering',
                                              'Undergraduate Business Admin'])
    return util.filter_and_balance_dataset(
            student_data, TASK_QUESTION_MAP[task], reduction_function=reduction_function,
            inclusion_function=inclusion_function, inclusion_only=True)

def get_task_outputs(task, balance_data=True):
    hashes_by_semester = util.get_hashes_by_semester(train=True, test=True)
    all_student_data = []
    for semester, hashes in hashes_by_semester.items():
        all_student_data.append(util.load_student_data_by_semester(
            semester, hashes, additional_surveys=True))
    student_data = pd.concat(all_student_data, ignore_index=True, sort=True)
    balanced_data = _filter_and_balance_dataset(student_data, task) if balance_data else student_data

    allowed_ids = set(balanced_data['sha1_hash'])

    student_outputs = []
    hash_to_output = {}
    for student_hash in allowed_ids:
        response = student_data[student_data.sha1_hash == student_hash][TASK_QUESTION_MAP[task]].values[0]
        output = map_to_bucket(task, response)
        if output is not None:
            student_outputs.append(output)
            hash_to_output[student_hash] = output
    return hash_to_output

def get_counts_by_task(task, print_results=False, balance_data=True):
    task_outputs = get_task_outputs(task, balance_data=balance_data).values()
    counts = Counter(task_outputs)
    if print_results:
        print(pd.Series(counts))
    return counts

In [ ]:
tasks = ['class_year', 'gender', 'school', 'sleep', 'gpa', 'depression_phq8_all_s4', 'depression_phq8_major_s4']

In [ ]:
counts_by_task = {}
for task in tasks:
    print(task)
    counts_by_task[task] = get_counts_by_task(task, print_results=True)
    print('\n\n')

In [ ]:
tasks = ['class_year', 'gender', 'school']

In [ ]:
# task counts before filtering
counts_by_task = {}
for task in tasks:
    print(task)
    counts_by_task[task] = get_counts_by_task(task, print_results=True, balance_data=False)
    print('\n\n')

In [ ]:
def get_n_classes(counts):
    return len(counts)

def get_percent_minority_class(counts):
    return 100 * min(counts.values()) / sum(counts.values())

def calc_baseline_macro_f1_majority_class(counts):
    majority_class = counts.most_common()[0][0]
    total = sum(counts.values())
    f1s = []
    for actual_class, count in counts.items():
        rec = 1 if actual_class == majority_class else 0
        if actual_class == majority_class:
            prec = count / total
        else:
            # technically nothing is predicted in this class so it is a divide by zero
            prec = 0
        if rec > 0 or prec > 0:
            f1 = 2 * (prec * rec) / (prec + rec)
        else:
            f1 = 0
        f1s.append(f1)
    return np.mean(f1s)

def calc_baseline_macro_f1_random_fold(counts):
    f1s = []
    sum_items = sum(counts.values())
    n_classes = len(counts)
    for actual_class, count in counts.items():
        prec = count / sum_items
        rec = 1 / n_classes
        f1s.append(2 * (prec * rec) / (prec + rec))
    return np.mean(f1s)

def calc_baseline_macro_f1_random(task):
    student_outputs = get_task_outputs(task, balance_data=True)
    fold_id_map = util.get_folds(10)
    all_f1s = []
    for fold in fold_id_map.values():
        counts = Counter()
        for student_hash, student_class in student_outputs.items():
            if student_hash in fold:
                counts[student_class] += 1
        f1_baseline = calc_baseline_macro_f1_random_fold(counts)
        all_f1s.append(f1_baseline)
    return np.mean(all_f1s) * 100

In [ ]:
n_classes = [get_n_classes(counts_by_task[task]) for task in tasks]
percent_minority = [get_percent_minority_class(counts_by_task[task]) for task in tasks]
n_instances = [sum(list(counts_by_task[task].values())) for task in tasks]

In [ ]:
data = {'Classes': n_classes, 'Instances': n_instances, 'Percent in Smallest Class': percent_minority}
df = pd.DataFrame(data, index=tasks)
print(df)

In [ ]:
def applytab(row):
    print('\t'.join(map(str, map(int,row.values))))
#print('\t'.join(map(str,df.columns))) # to print the column names if required
print(df.apply(applytab,axis=1))


In [ ]:
for task in tasks:
    print(task)
    print("majority:", round(calc_baseline_macro_f1_majority_class(counts_by_task[task]), 2))
    print("random:", round(calc_baseline_macro_f1_random(task), 2))

## Surface Level

In [ ]:
# does not actually matter what the vectors are, just using this code to get the sequences
whitelist = 'data/building_whitelists/location_twitter_reddit_mention_campus_info.whitelist'
twitter_geo_embeddings, _ = get_location_embeddings(
    use_whitelist=whitelist, only_text_embeddings=True,
    text_embedding_methods=['twitter_geo.tfidf_pretrained.glove_twitter'])

vector_map, sequence_map = prep_embeddings(twitter_geo_embeddings)

vector_list = [v for k, v in vector_map.items()]
sequence_list = [v for k, v in sequence_map.items()]

### LocPres

In [ ]:
min_vals_per_class = 100
location_whitelist = _build_location_whitelist(sequence_map, min_vals_per_class)

f1s = []
for fold_ids in fold_id_map.values():
    test_vector_list = [v for k, v in vector_map.items() if k in fold_ids]
    test_sequence_list = [v for k, v in sequence_map.items() if k in fold_ids]
    build_f1s = []
    for building in location_whitelist:
        _, classes = create_input_output_location_presence(test_sequence_list, test_vector_list, building)
        class_counts_for_building = Counter(classes)
        build_f1s.append(calc_baseline_macro_f1_random_fold(class_counts_for_building))
    f1s.append(np.mean(build_f1s))
print("random:", round(np.mean(f1s), 2) * 100)

### FuncPres

In [ ]:
# need functionality whitelist, etc., as input
min_vals_per_class = 100
building_functionality_map, functionalities, buildings = util.get_functionalities()
functionality_whitelist = _build_functionality_whitelist(
    sequence_map, min_vals_per_class, building_functionality_map, functionalities)

func_to_buildings = defaultdict(list)
for functionality in functionality_whitelist:
    for building_name, fns in building_functionality_map.items():
        if functionality in fns:
            func_to_buildings[functionality].append(building_name)


f1s = []
for fold_ids in fold_id_map.values():
    test_vector_list = [v for k, v in vector_map.items() if k in fold_ids]
    test_sequence_list = [v for k, v in sequence_map.items() if k in fold_ids]
    func_f1s = []
    for functionality in functionality_whitelist:
        _, classes = create_input_output_functionality_presence(test_sequence_list, test_vector_list, functionality, 
                                                                func_to_buildings)
        class_counts_for_building = Counter(classes)
        func_f1s.append(calc_baseline_macro_f1_random_fold(class_counts_for_building))
    f1s.append(np.mean(func_f1s))
print("random:", round(np.mean(f1s), 2) * 100)